In [1]:
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
seed = 0
np.random.seed(seed)

## Fetch the jet tagging dataset from Open ML (run this only once, this can take a while)

In [2]:
data = fetch_openml('hls4ml_lhc_jets_hlf')
X, y = data['data'], data['target']

### Let's print some information about the dataset
Print the feature names and the dataset shape

In [3]:
print(data['feature_names'])
print(X.shape, y.shape)
print(X[:5])
print(y[:10])

['zlogz', 'c1_b0_mmdt', 'c1_b1_mmdt', 'c1_b2_mmdt', 'c2_b1_mmdt', 'c2_b2_mmdt', 'd2_b1_mmdt', 'd2_b2_mmdt', 'd2_a1_b1_mmdt', 'd2_a1_b2_mmdt', 'm2_b1_mmdt', 'm2_b2_mmdt', 'n2_b1_mmdt', 'n2_b2_mmdt', 'mass_mmdt', 'multiplicity']
(830000, 16) (830000,)
[[-2.93512535e+00  3.83155316e-01  5.12587558e-03  8.42466834e-05
   9.06995591e-03  1.78931368e-04  1.76944518e+00  2.12389827e+00
   1.76944518e+00  3.08185428e-01  1.35686919e-01  8.32780078e-02
   4.12136108e-01  2.99057871e-01  8.92688179e+00  7.50000000e+01]
 [-1.92733514e+00  2.70698756e-01  1.58540264e-03  1.13709866e-05
   3.23237223e-03  2.91449633e-05  2.03883362e+00  2.56309891e+00
   2.03883362e+00  2.11886495e-01  6.37285784e-02  3.63104008e-02
   3.10216516e-01  2.26661310e-01  3.88651156e+00  3.10000000e+01]
 [-3.11214662e+00  4.58171129e-01  9.79138538e-02  2.85884105e-02
   1.24277540e-01  3.84868123e-02  1.26925385e+00  1.34623826e+00
   1.26925385e+00  2.46488109e-01  1.15635969e-01  7.90941268e-02
   3.57558519e-01  2.8

As you saw above, the `y` target is an array of strings, e.g. \['g', 'w',...\] etc.
We need to make this a "One Hot" encoding for the training.
Then, split the dataset into training and validation sets

In [4]:
le = LabelEncoder()
le.fit(y)
print(le.classes_)

['g' 'q' 't' 'w' 'z']


In [5]:
yl = le.transform(y)
print(yl[:10])

[0 3 2 4 3 3 2 0 4 0]


In [6]:
yc = to_categorical(yl, len(le.classes_))
print(yc[:10])

[[1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]]


In [7]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, yc, test_size=0.2, random_state=42)

In [8]:
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import accuracy_score
import setGPU

setGPU: Setting GPU to: 4


In [9]:
def History(label, h):
    plt.plot( h.history['loss'], label ='Training Loss: {}'.format(label))
    plt.plot( h.history['val_loss'], label = 'Validation Loss: {}'.format(label))
    plt.plot( h.history['accuracy'], label = 'Training Accuracy: {}'.format(label))
    plt.plot( h.history['val_accuracy'], label = 'Validation Accuracy: {}'.format(label))
    plt.legend()
    plt.show()

In [10]:
def Accuracy(m):
    pred = m.predict( X_test)
    a = accuracy_score(np.argmax(y_test, axis=1), np.argmax(pred, axis=1))
    print("Accuracy: {}: {}".format(m.name, a))
    return pred, a

## Construct a simple dense model

We'll use 3 hidden layers with 32, then 64, then 64 neurons. Each layer will use `relu` activation.
Add an output layer with 5 neurons (one for each class), then finish with Softmax activation.

In [11]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Dense, Activation, BatchNormalization, Conv1D, Dropout, Reshape, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1

In [12]:
i = Input(shape=(16,))
d = Dense(32, name='fc1', activation='relu')(i)
d = Dense(64, name='fc2', activation='relu')(d)
d = Dense(64, name='fc3', activation='relu')(d)
o = Dense(len(le.classes_), activation='softmax', name='output', kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001))(d)
model_1 = Model(inputs=i, outputs=o, name='model_DNN')

In [13]:
model_1.summary()

Model: "model_DNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16)]              0         
_________________________________________________________________
fc1 (Dense)                  (None, 32)                544       
_________________________________________________________________
fc2 (Dense)                  (None, 64)                2112      
_________________________________________________________________
fc3 (Dense)                  (None, 64)                4160      
_________________________________________________________________
output (Dense)               (None, 5)                 325       
Total params: 7,141
Trainable params: 7,141
Non-trainable params: 0
_________________________________________________________________


## Train the model
We'll use Adam optimizer with categorical crossentropy loss.
The model isn't very complex, so this should just take a few minutes even on the CPU.

In [14]:
adam = Adam(lr=0.0001)
model_1.compile(optimizer=adam, loss=['categorical_crossentropy'], metrics=['accuracy'])

In [15]:
model_1.fit(X_train_val, y_train_val, batch_size=1024,
              epochs=2, validation_split=0.25, shuffle=True)

Epoch 1/2
487/487 [==============================] - 1s 3ms/step - loss: 1.8687 - accuracy: 0.3806 - val_loss: 1.3102 - val_accuracy: 0.4289
Epoch 2/2
487/487 [==============================] - 1s 2ms/step - loss: 1.2659 - accuracy: 0.4822 - val_loss: 1.2186 - val_accuracy: 0.5288


In [16]:
y_predict_1, a_1 = Accuracy(model_1)

Accuracy: model_DNN: 0.5287048192771084


## Construct a simple CNN model

In [17]:
i = Input(shape=(16,))
r = Reshape((16,1))(i)
c = Conv1D(16, 3, name='cv1', activation='relu')(r)
c = Conv1D(16, 3, name='cv2', activation='relu')(c)
c = Conv1D(16, 3, name='cv3', activation='relu')(c)
f = Flatten()(c)
o = Dense(len(le.classes_), activation='softmax', name='output', kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001))(f)
model_2 = Model(inputs=i, outputs=o, name='model_CNN')

In [18]:
model_2.summary()

Model: "model_CNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 16)]              0         
_________________________________________________________________
reshape (Reshape)            (None, 16, 1)             0         
_________________________________________________________________
cv1 (Conv1D)                 (None, 14, 16)            64        
_________________________________________________________________
cv2 (Conv1D)                 (None, 12, 16)            784       
_________________________________________________________________
cv3 (Conv1D)                 (None, 10, 16)            784       
_________________________________________________________________
flatten (Flatten)            (None, 160)               0         
_________________________________________________________________
output (Dense)               (None, 5)                 80

## Train the model
We'll use Adam optimizer with categorical crossentropy loss.
The model isn't very complex, so this should just take a few minutes even on the CPU.

In [19]:
adam = Adam(lr=0.0001)
model_2.compile(optimizer=adam, loss=['categorical_crossentropy'], metrics=['accuracy'])

In [20]:
model_2.fit(X_train_val, y_train_val, batch_size=1024,
              epochs=2, validation_split=0.25, shuffle=True)

Epoch 1/2
487/487 [==============================] - 2s 4ms/step - loss: 1.4583 - accuracy: 0.3559 - val_loss: 1.2897 - val_accuracy: 0.4795
Epoch 2/2
487/487 [==============================] - 2s 3ms/step - loss: 1.2041 - accuracy: 0.5502 - val_loss: 1.1561 - val_accuracy: 0.5681


In [21]:
y_predict_2, a_2 = Accuracy(model_2)

Accuracy: model_CNN: 0.5687951807228916


## What is the best model then ?

In [22]:
print("the best model is: {}".format(model_1.name if a_1 > a_2 else model_2.name))

the best model is: model_CNN
